In [1]:
%run hyper-elasticity_debug.py

[[0.]
 [0.]
 [1.]]
[[0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]]
[[1. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
  1.69e-01
  7.58e-02
  1.07e-02
  3.43e-04
  7.44e-07


In [2]:
whos

Variable       Type        Data/Info
------------------------------------
DbarF          ndarray     3x3x3x3x1: 81 elems, type `float64`, 648 bytes
F              ndarray     3x3x3x3x1: 81 elems, type `float64`, 648 bytes
Fn             float64     6.0
G              function    <function <lambda> at 0x14e528a7ea60>
G_K_dF         function    <function <lambda> at 0x14e528a7eb80>
Ghat4          ndarray     3x3x3x3x3x3x1: 729 elems, type `float64`, 5832 bytes
I              ndarray     3x3x3x3x1: 81 elems, type `float64`, 648 bytes
I4             ndarray     3x3x3x3x3x3x1: 729 elems, type `float64`, 5832 bytes
I4rt           ndarray     3x3x3x3x3x3x1: 729 elems, type `float64`, 5832 bytes
I4s            ndarray     3x3x3x3x3x3x1: 729 elems, type `float64`, 5832 bytes
II             ndarray     3x3x3x3x3x3x1: 729 elems, type `float64`, 5832 bytes
K              ndarray     3x3x1: 9 elems, type `float64`, 72 bytes
K4             ndarray     3x3x3x3x3x3x1: 729 elems, type `float64`, 5832 b

In [13]:
tmp_1 = fft(F)
tmp_1.shape

(3, 3, 3, 3, 1)

In [16]:
Ghat4.shape

(3, 3, 3, 3, 3, 3, 1)

In [17]:
help(G)

Help on function <lambda> in module __main__:

<lambda> lambda A2
    # functions for the projection 'G', and the product 'G : K^LT : (delta F)^T'



In [18]:
G??

In [20]:
tmp_2 = ddot42(Ghat4, tmp_1)
tmp_2.shape

(3, 3, 3, 3, 1)

In [21]:
tmp_3 = ifft(tmp_2)
tmp_3.shape

(3, 3, 3, 3, 1)

In [22]:
tmp_3.reshape(-1).shape

(81,)

In [24]:
%less hyper-elasticity_debug.py

In [25]:
who

DbarF	 F	 Fn	 G	 G_K_dF	 Ghat4	 I	 I4	 I4rt	 
I4s	 II	 K	 K4	 K_dF	 Nx	 Ny	 Nz	 P	 
Q	 Z	 b	 constitutive	 dFm	 ddot42	 ddot44	 delta	 dot11	 
dot22	 dot24	 dot42	 dyad22	 fft	 freq	 i	 ifft	 iiter	 
itertools	 j	 l	 m	 mu	 ndim	 norm	 np	 param	 
phase	 q	 shape	 sp	 tmp_1	 tmp_2	 tmp_3	 trans2	 x	 



In [29]:
mu.shape

(3, 3, 1)

In [ ]:
q